# Implement a physically correct distribution

In order to understand lubrication better, we simulate thin layers of lubricant on a metallic surface, solvated in water.
Different structures of lubricant films are created by varying parameters like their concentration and the charge of the surface.
The lubricant is somewhat solvable in water, thus parts of the film will diffuse into the bulk water.
Lubricant molecules are charged, and their distribution is described by the Poisson-Bolzmann equation:
$
\begin{align}
\Psi(x) &= \frac{2kT}{e} \log\Big(\frac{1 + \gamma e^{-\kappa x}}{1- \gamma e^{-\kappa x}}\Big) 
        \approx \frac{4kT}{e} \gamma e^{-\kappa x} \\
c_{Na}(x) &= c_{Na}(\infty) e^{-e \Psi(x)/kT}
\end{align}
$

In [7]:
import numpy
import pandas
import generate_structure
import matplotlib.pyplot as plt

from IPython.display import Image
from IPython.core.display import HTML 

In [9]:
Image(url= "./z.png",  width=500)